In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

In [ ]:
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with th

In [ ]:
#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
import pandas as pd

docs = pd.read_csv('pages-export.csv')
pd.DataFrame(docs['Text Only Transcript']).head(5)

,Text Only Transcript
0,Sept. travling in company with Elder [[Benjami...
1,"September 11, 1836 ~ Sunday\n\n\nSept. ^11th^ ..."
2,"their Oxen, their Horses their Tents their arm..."
3,{[On] the [illegible] of Sister [[Mary Barker|...
4,"September 22, 1836 ~ Thursday\n\n\nSept 22 Pre..."


In [ ]:
mask = docs['Text Only Transcript'].str.len() >=1
docs = docs.loc[mask]
docs['Text Only Transcript'] = docs['Text Only Transcript'].astype('str')

In [ ]:
docs.head(3)

,Internal ID,Document Type,Parent ID,Order,Parent Name,UUID,Name,Website URL,Short URL,Image URL,Original Transcript,Text Only Transcript,People,Places,First Date,Dates,Topics
0,1,Journals,665.0,102,"Journal (December 29, 1833 – January 3, 1838)",92e144b8-d10c-4d42-9616-bcd81375c947,page_0001,https://wilfordwoodruffpapers.org/documents/65...,https://wilfordwoodruffpapers.org/p/jR,https://wilford-woodruff-papers.nyc3.digitaloc...,<p></p><p>Sept. travling in company with Elder...,Sept. travling in company with Elder [[Benjami...,Isaiah Benton|Jefferson Turnbow|Lucinda Benton...,"Calloway County, Kentucky",NaN,1836-09-10,testimony|Elder|truth|called|Doctrine and Cove...
1,2,Journals,665.0,103,"Journal (December 29, 1833 – January 3, 1838)",ded69d92-7819-49ad-9677-e525af60a58c,page_0002,https://wilfordwoodruffpapers.org/documents/65...,https://wilfordwoodruffpapers.org/p/k5,https://wilford-woodruff-papers.nyc3.digitaloc...,"<p></p><p><strong><time datetime=""1836-09-11"">...","September 11, 1836 ~ Sunday\n\n\nSept. ^11th^ ...",Abraham Owen Smoot|Daniel Cathcart|David Wyman...,Missouri|Zion,NaN,1836-09-11|1836-09-12|1836-09-13|1836-09-14|18...,God|laying on of hands|Zion|last days|healed|p...
2,3,Journals,665.0,104,"Journal (December 29, 1833 – January 3, 1838)",291ca375-cd7a-4280-a11a-aa002b6b4a82,page_0003,https://wilfordwoodruffpapers.org/documents/65...,https://wilfordwoodruffpapers.org/p/l5,https://wilford-woodruff-papers.nyc3.digitaloc...,"<p></p><p>their Oxen, their Horses their Tents...","their Oxen, their Horses their Tents their arm...",Albert Petty|Daniel Cathcart|David Wyman Patte...,"Zion|Cypress Creek, Benton County, Tennessee",NaN,NaN,baptism|God|Zion|Elder|baptized|children of Is...


In [ ]:
doc_list = list(docs['Text Only Transcript'])
doc_list[1091]

"October 17, 1835 ~ Saturday\n\n\n17th Oct Preached at Mr [[Benjamin Lynn Clapp|Clapps]] &amp; Baptised 1 one\nthen Rode to Br [[Alfred Loy|Loys]] Preached at his house\nand returned to Br [[Benjamin Lynn Clapp|Clapp's]]. 3 miles\n\n\nOctober 18, 1835 ~ Sunday\n\n\n18 On the Lords day Rode to [[Clarks River, Calloway County, Kentucky|Clarks River]] 10 mi\n&amp; Preached in the meeting house had A \ndebate with A Campbellite [[CBI|Priest]]. Also Preached\nat night in the same place on the Authenticity\nof the [[Book of Mormon|Book of Mormon]] to A large Assembly\n\n\nOctober 19, 1835 ~ Monday\n\n\n19th Procured 1 subscriber for [[Latter Day Saints|M[essenger] &amp; Ad[vocate].]] Preached\nin the evening at Br [[Daniel Stillwell Thomas|Thomas'es]] the [[spirit of God|Sp^i^rit of God]] rested {upon me}.\n\n\nOctober 20, 1835 ~ Tuesday\n\n\n20th I Baptized 3 [[Campbellite|Campbellite]]s one A deacon\nthen Rode 12 miles &amp; Preached at Mr [[Joseph Greenwood|Joseph\nGreenwoods]] Mr [[Josep

In [ ]:
# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(doc_list, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/912 [00:00<?, ?it/s]

In [ ]:
import pickle
with open("my-embeddings.pkl", "wb") as fOut:
    pickle.dump({'embeddings': corpus_embeddings},fOut)

In [ ]:
def search(query):
    print("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, doc_list[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
        
    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    results = []
    for hit in hits[0:3]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], doc_list[hit['corpus_id']].replace("\n", " ")))
        results.append(doc_list[hit['corpus_id']])
        
    return results
        


In [ ]:
results = search("Who is God?")

Input question: Who is God?

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	1.912	of the Almighty.  Go then to the beginning that you may understand. I ask this congregation what kind of a being is God? Turn your thoughts in your own hearts and say have any of you seen or heard Him or communed with Him. This is a question which may occupy your attention. The scriptures informs us that this is eternal life to know thee the only true and living God and Jesus Christ whom thou hast sent. If any enquire What kind of a being God is I would say if you do not know Him you have not eternal life Go back and find out what kind of a being God is. If I am the man that shows you what kind of being  God is then let every man and woman sit in silence and never lift up the hand against me again. If I do not this I will make no more pretentions to inspiration or to be a Prophet. I should be like the rest of the world a false teacher and you would want to take my life. But you might just 

In [ ]:
df_results = pd.DataFrame(results, columns = ['Text Only Transcript'])
docs_match = docs.merge(df_results, on='Text Only Transcript')
docs_match

,Internal ID,Document Type,Parent ID,Order,Parent Name,UUID,Name,Website URL,Short URL,Image URL,Original Transcript,Text Only Transcript,People,Places,First Date,Dates,Topics
0,23931,NaN,NaN,36,NaN,c0083255-3480-428b-98a0-ba2bc1a5117f,page_0036,NaN,https://wilfordwoodruffpapers.org/p/XnVg,https://wilford-woodruff-papers.nyc3.digitaloc...,<p>of the Almighty.</p>\n\n<p>Go then to the b...,of the Almighty.\n\nGo then to the beginning t...,NaN,NaN,NaN,NaN,NaN
1,26689,Autobiographies,42658.0,36,Autobiography 1883 Tullidge's Quarterly Magazi...,abae9846-317f-4d1c-8d0a-63f367ffbcd6,page_0036,https://wilfordwoodruffpapers.org/documents/10...,https://wilfordwoodruffpapers.org/p/j5DW,https://wilford-woodruff-papers.nyc3.digitaloc...,<p>of the Almighty.</p>\n\n<p>Go then to the b...,of the Almighty.\n\nGo then to the beginning t...,Joseph Smith Jr.|Adam (OT),NaN,1844-04-06,NaN,NaN
2,29228,Additional,42564.0,398,"Religious-Personal Bible with Study Notes, 1830",45d09f7a-4187-4829-a5b3-ad22fda9ac35,page_0048,https://wilfordwoodruffpapers.org/documents/a8...,https://wilfordwoodruffpapers.org/p/RBOw,https://wilford-woodruff-papers.nyc3.digitaloc...,<p>God is with the Latter Day Saints {shorthan...,God is with the Latter Day Saints {shorthand},NaN,NaN,NaN,NaN,NaN
